# Outline for Wednesday, April 14
## Databases 1 - SQL

You will be able to:
 - Explain how a database is different from a CSV file or a JSON file
 - Use SQLite to connect to a database and pandas to query it
 - Write basic queries on a database using SELECT, FROM, WHERE, ORDER BY, and LIMIT
 
Vocabulary:
 - query
 - schema
 
Useful modules:
 - sqlite3

## What aren't we going to cover?

 - Schema design. (Changing the tables/columns and the relationships between them.)
 - Adding/removing rows from a database.
 - Concurrency. (Complications from multiple people querying the database at the same time.)
 - Performance.
 - Joins. (Combining multiple tables with related information.)
 
We are treating the database as a fixed object and giving you the tools necessary to interact with it.

## Databases and SQL

SQL == Structured Query Language

What is a query?
 - Request for specific data from a database
 - Like a function call - query will have some parameter-like pieces that say what we want, and the query will return the data that we are looking for
 
Why use a database rather than CSV/JSON?
Database properties:
 - Multiple named tables
 - Columns always named
 - Enforced data types for each column (CSV had everything as a string)
 - Multiple users can write to the database at the same time
 - Supports queries
 - Steeper learning curve
 
Popular SQL Databases (Mostly work the same)
 - SQL Server
 - Oracle
 - MySQL
 - PostgreSQL
 - SQLite <-- We will use this one because you already have the sqlite3 module

# Exercise: Madison bus routes in database form!

You will need to download the bus.db file and put it in the same directory as your notebook

## Database connection

In [12]:
import sqlite3
import pandas as pd
import os

## Typo example - bussssss.db

In [9]:
conn = sqlite3.connect("busssss.db") #connect takes one argument: a filename

### pd.read_sql("QUERY", CONNECTION)
* Query: SELECT * FROM sqlite_master

In [10]:
# read_sql has two parameters: QUERY (str), connection (made with sqlite3.connect())
# SQL is (mostly) not case sensitive
df = pd.read_sql("select * from sqlite_master", conn)
df

,type,name,tbl_name,rootpage,sql


### Let's close this bad connection

In [11]:
conn.close()

### Now let's open the correct bus.db

In [13]:
path = "bus.db"
assert os.path.exists(path)
conn = sqlite3.connect(path)
conn

In [14]:
df = pd.read_sql("select * from sqlite_master", conn)
df

,type,name,tbl_name,rootpage,sql
0,table,boarding,boarding,2,"CREATE TABLE ""boarding"" (\n""index"" INTEGER,\n ..."
1,index,ix_boarding_index,boarding,3,"CREATE INDEX ""ix_boarding_index""ON ""boarding"" ..."
2,table,routes,routes,55,"CREATE TABLE ""routes"" (\n""index"" INTEGER,\n ""..."
3,index,ix_routes_index,routes,57,"CREATE INDEX ""ix_routes_index""ON ""routes"" (""in..."


### Extract sql column from the DataFrame

In [15]:
df["sql"]

0    CREATE TABLE "boarding" (\n"index" INTEGER,\n ...
1    CREATE INDEX "ix_boarding_index"ON "boarding" ...
2    CREATE TABLE "routes" (\n"index" INTEGER,\n  "...
3    CREATE INDEX "ix_routes_index"ON "routes" ("in...
Name: sql, dtype: object

### Let's iterate over df["sql"]

In [16]:
for code in df["sql"]:
    print(code)

CREATE TABLE "boarding" (
"index" INTEGER,
  "StopID" INTEGER,
  "Route" INTEGER,
  "Lat" REAL,
  "Lon" REAL,
  "DailyBoardings" REAL
)
CREATE INDEX "ix_boarding_index"ON "boarding" ("index")
CREATE TABLE "routes" (
"index" INTEGER,
  "OBJECTID" INTEGER,
  "trips_routes_route_id" INTEGER,
  "route_short_name" INTEGER,
  "route_url" TEXT,
  "ShapeSTLength" REAL
)
CREATE INDEX "ix_routes_index"ON "routes" ("index")


### Query: SELECT * FROM \<table_name\>

In [17]:
pd.read_sql("SELECT * FROM routes", conn)

,index,OBJECTID,trips_routes_route_id,route_short_name,route_url,ShapeSTLength
0,0,63,8052,1,http://www.cityofmadison.com/Metro/schedules/R...,32379.426524
1,1,64,8053,2,http://www.cityofmadison.com/Metro/schedules/R...,96906.965571
2,2,65,8054,3,http://www.cityofmadison.com/Metro/schedules/R...,76436.645644
3,3,66,8055,4,http://www.cityofmadison.com/Metro/schedules/R...,64774.133485
4,4,67,8056,5,http://www.cityofmadison.com/Metro/schedules/R...,61216.722662
...,...,...,...,...,...,...
57,57,120,8109,78,http://www.cityofmadison.com/Metro/schedules/R...,95826.277218
58,58,121,8110,80,http://www.cityofmadison.com/Metro/schedules/R...,31831.761009
59,59,122,8111,81,http://www.cityofmadison.com/Metro/schedules/R...,26536.800591
60,60,123,8112,82,http://www.cityofmadison.com/Metro/schedules/R...,23287.980173


In [18]:
df = pd.read_sql("SELECT * FROM routes", conn)
df.head(10)

,index,OBJECTID,trips_routes_route_id,route_short_name,route_url,ShapeSTLength
0,0,63,8052,1,http://www.cityofmadison.com/Metro/schedules/R...,32379.426524
1,1,64,8053,2,http://www.cityofmadison.com/Metro/schedules/R...,96906.965571
2,2,65,8054,3,http://www.cityofmadison.com/Metro/schedules/R...,76436.645644
3,3,66,8055,4,http://www.cityofmadison.com/Metro/schedules/R...,64774.133485
4,4,67,8056,5,http://www.cityofmadison.com/Metro/schedules/R...,61216.722662
5,5,68,8057,6,http://www.cityofmadison.com/Metro/schedules/R...,151142.298370
6,6,69,8058,7,http://www.cityofmadison.com/Metro/schedules/R...,98617.005665
7,7,70,8059,8,http://www.cityofmadison.com/Metro/schedules/R...,56732.757385
8,8,71,8060,10,http://www.cityofmadison.com/Metro/schedules/R...,113468.940882
9,9,72,8061,11,http://www.cityofmadison.com/Metro/schedules/R...,131900.584375


### Adding LIMIT to the query
* SELECT * FROM \<table_name\> LIMIT \<num_of_rows\>

In [19]:
df = pd.read_sql("SELECT * FROM routes LIMIT 10", conn)
df

,index,OBJECTID,trips_routes_route_id,route_short_name,route_url,ShapeSTLength
0,0,63,8052,1,http://www.cityofmadison.com/Metro/schedules/R...,32379.426524
1,1,64,8053,2,http://www.cityofmadison.com/Metro/schedules/R...,96906.965571
2,2,65,8054,3,http://www.cityofmadison.com/Metro/schedules/R...,76436.645644
3,3,66,8055,4,http://www.cityofmadison.com/Metro/schedules/R...,64774.133485
4,4,67,8056,5,http://www.cityofmadison.com/Metro/schedules/R...,61216.722662
5,5,68,8057,6,http://www.cityofmadison.com/Metro/schedules/R...,151142.298370
6,6,69,8058,7,http://www.cityofmadison.com/Metro/schedules/R...,98617.005665
7,7,70,8059,8,http://www.cityofmadison.com/Metro/schedules/R...,56732.757385
8,8,71,8060,10,http://www.cityofmadison.com/Metro/schedules/R...,113468.940882
9,9,72,8061,11,http://www.cityofmadison.com/Metro/schedules/R...,131900.584375


# How many people get on a bus in Madison every day?
- we are interested in boarding table to answer this question

In [21]:
# We want DailyBoardings from boarding
qry = "SELECT DailyBoardings FROM boarding"
df  = pd.read_sql(qry, conn)
daily_riders = df["DailyBoardings"]
daily_riders.sum()

55987.18

In [22]:
qry2 = "SELECT SUM(DailyBoardings) FROM boarding"
pd.read_sql(qry2, conn)

,SUM(DailyBoardings)
0,55987.18


# Go West - which bus should I take to go as far west as possible?
- Smallest Longitude

In [26]:
query = "SELECT Route,Lon FROM boarding"
df = pd.read_sql(query, conn)
df.loc[df["Lon"] == df["Lon"].min()]

,Route,Lon
3489,55,-89.564243
3490,75,-89.564243


In [27]:
query2 = "SELECT * FROM boarding ORDER BY Lon ASC LIMIT 3"

pd.read_sql(query2, conn)

,index,StopID,Route,Lat,Lon,DailyBoardings
0,3489,4400,55,42.995476,-89.564243,59.31
1,3490,4400,75,42.995476,-89.564243,168.50
2,3529,4559,55,42.987673,-89.553340,0.92


In [28]:
conn.close()